# Blaze SQL Query Engine - Python Integration

This notebook demonstrates how to use the Blaze SQL query engine with Python,
including PyArrow integration, Pandas/Polars conversion, and visualization.

## Prerequisites

```bash
pip install pyarrow pandas polars matplotlib
pip install blaze-sql  # When Python bindings are available
```

## Setup

First, let's import the necessary libraries and create a connection to Blaze.

In [ ]:
import pyarrow as pa
import pyarrow.ipc as ipc
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import numpy as np

# When Blaze Python bindings are available:
# from blaze import Connection

print(f"PyArrow version: {pa.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Polars version: {pl.__version__}")

## Creating Sample Data

Let's create some sample data using PyArrow and register it with Blaze.

In [ ]:
# Create sample sales data
np.random.seed(42)
n_rows = 1000

sales_data = pa.table({
    'order_id': pa.array(range(1, n_rows + 1)),
    'product': pa.array(np.random.choice(['Widget', 'Gadget', 'Gizmo', 'Thing'], n_rows)),
    'quantity': pa.array(np.random.randint(1, 20, n_rows)),
    'price': pa.array(np.random.uniform(10, 500, n_rows).round(2)),
    'region': pa.array(np.random.choice(['North', 'South', 'East', 'West'], n_rows)),
    'date': pa.array(pd.date_range('2024-01-01', periods=n_rows, freq='H'))
})

print(f"Created table with {sales_data.num_rows} rows")
print(f"Schema: {sales_data.schema}")
sales_data.slice(0, 5).to_pandas()

In [ ]:
# Create customer data
customers_data = pa.table({
    'customer_id': pa.array(range(1, 101)),
    'name': pa.array([f'Customer_{i}' for i in range(1, 101)]),
    'region': pa.array(np.random.choice(['North', 'South', 'East', 'West'], 100)),
    'tier': pa.array(np.random.choice(['Bronze', 'Silver', 'Gold', 'Platinum'], 100))
})

print(f"Created customers table with {customers_data.num_rows} rows")
customers_data.slice(0, 5).to_pandas()

## Simulated Blaze Queries

The following cells show example queries that would work with Blaze.
For now, we'll simulate the results using Pandas/Polars.

In [ ]:
# Convert to Pandas for simulation
sales_df = sales_data.to_pandas()
sales_df['revenue'] = sales_df['quantity'] * sales_df['price']

# Simulating: SELECT region, SUM(quantity * price) as revenue FROM sales GROUP BY region
region_revenue = sales_df.groupby('region')['revenue'].sum().reset_index()
print("Regional Revenue Summary:")
print(region_revenue)

In [ ]:
# Using Polars for fast analytics
sales_pl = pl.from_arrow(sales_data)

# Simulating: SELECT product, AVG(price), COUNT(*) FROM sales GROUP BY product ORDER BY COUNT(*) DESC
product_stats = (
    sales_pl
    .group_by('product')
    .agg([
        pl.col('price').mean().alias('avg_price'),
        pl.len().alias('order_count'),
        (pl.col('quantity') * pl.col('price')).sum().alias('total_revenue')
    ])
    .sort('order_count', descending=True)
)

print("Product Statistics:")
print(product_stats)

## Visualization Examples

In [ ]:
# Bar chart of regional revenue
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Regional revenue bar chart
ax1 = axes[0]
colors = ['#2ecc71', '#3498db', '#9b59b6', '#e74c3c']
ax1.bar(region_revenue['region'], region_revenue['revenue'], color=colors)
ax1.set_xlabel('Region')
ax1.set_ylabel('Revenue ($)')
ax1.set_title('Revenue by Region')
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Product distribution pie chart
ax2 = axes[1]
product_counts = sales_df['product'].value_counts()
ax2.pie(product_counts.values, labels=product_counts.index, autopct='%1.1f%%', colors=colors)
ax2.set_title('Order Distribution by Product')

plt.tight_layout()
plt.show()

In [ ]:
# Time series analysis
sales_df['date'] = pd.to_datetime(sales_df['date'])
daily_revenue = sales_df.set_index('date').resample('D')['revenue'].sum()

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(daily_revenue.index, daily_revenue.values, color='#3498db', linewidth=2)
ax.fill_between(daily_revenue.index, daily_revenue.values, alpha=0.3, color='#3498db')
ax.set_xlabel('Date')
ax.set_ylabel('Daily Revenue ($)')
ax.set_title('Daily Revenue Trend')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Working with Arrow IPC Format

Blaze returns results in Arrow IPC format for efficient data transfer.

In [ ]:
# Serialize table to Arrow IPC (simulating Blaze output)
sink = pa.BufferOutputStream()
with ipc.new_stream(sink, sales_data.schema) as writer:
    writer.write_table(sales_data)
arrow_buffer = sink.getvalue()

print(f"Arrow IPC buffer size: {len(arrow_buffer):,} bytes")

# Read back from Arrow IPC (simulating reading Blaze results)
reader = ipc.open_stream(arrow_buffer)
result_table = reader.read_all()
print(f"Read back {result_table.num_rows} rows")
result_table.slice(0, 3).to_pandas()

## Window Functions Example

In [ ]:
# Simulating window functions with Polars
# This would be: SELECT *, ROW_NUMBER() OVER (PARTITION BY region ORDER BY revenue DESC) as rank

ranked_sales = (
    sales_pl
    .with_columns([
        (pl.col('quantity') * pl.col('price')).alias('revenue')
    ])
    .with_columns([
        pl.col('revenue')
          .rank(method='ordinal', descending=True)
          .over('region')
          .alias('region_rank')
    ])
    .filter(pl.col('region_rank') <= 3)
    .sort(['region', 'region_rank'])
)

print("Top 3 orders by revenue in each region:")
print(ranked_sales.select(['order_id', 'product', 'region', 'revenue', 'region_rank']))

## Integration with Blaze (When Available)

Once Blaze Python bindings are installed, you can use them like this:

```python
from blaze import Connection

# Create connection
conn = Connection.in_memory()

# Register Arrow table
conn.register_arrow('sales', sales_data)
conn.register_arrow('customers', customers_data)

# Execute query and get Arrow result
result = conn.query_arrow('''
    SELECT 
        region,
        COUNT(*) as order_count,
        SUM(quantity * price) as total_revenue,
        AVG(price) as avg_price
    FROM sales
    GROUP BY region
    ORDER BY total_revenue DESC
''')

# Convert to Pandas
df = result.to_pandas()
print(df)

# Or convert to Polars
pl_df = pl.from_arrow(result)
print(pl_df)
```

## Complex Query Examples

Here are some complex SQL queries that Blaze supports:

In [ ]:
# Example queries (as strings for reference)
queries = {
    "CTE with aggregation": """
        WITH daily_totals AS (
            SELECT 
                DATE(date) as order_date,
                region,
                SUM(quantity * price) as revenue
            FROM sales
            GROUP BY DATE(date), region
        )
        SELECT 
            order_date,
            region,
            revenue,
            SUM(revenue) OVER (
                PARTITION BY region 
                ORDER BY order_date 
                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
            ) as running_total
        FROM daily_totals
        ORDER BY region, order_date
    """,
    
    "Subquery in WHERE": """
        SELECT *
        FROM sales
        WHERE price > (
            SELECT AVG(price) * 1.5
            FROM sales
        )
    """,
    
    "CASE with window function": """
        SELECT 
            product,
            price,
            CASE 
                WHEN price >= (SELECT PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY price) FROM sales) THEN 'Premium'
                WHEN price >= (SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY price) FROM sales) THEN 'Standard'
                ELSE 'Budget'
            END as price_tier,
            RANK() OVER (PARTITION BY product ORDER BY price DESC) as price_rank
        FROM sales
    """
}

for name, query in queries.items():
    print(f"\n{'='*60}")
    print(f"Query: {name}")
    print('='*60)
    print(query.strip())

## Performance Comparison

In [ ]:
import time

# Create larger dataset for benchmarking
large_n = 100_000
large_data = pa.table({
    'id': pa.array(range(large_n)),
    'value': pa.array(np.random.uniform(0, 1000, large_n)),
    'category': pa.array(np.random.choice(['A', 'B', 'C', 'D'], large_n))
})

# Pandas timing
large_df = large_data.to_pandas()
start = time.time()
result_pd = large_df.groupby('category')['value'].agg(['sum', 'mean', 'count'])
pandas_time = time.time() - start

# Polars timing
large_pl = pl.from_arrow(large_data)
start = time.time()
result_pl = large_pl.group_by('category').agg([
    pl.col('value').sum(),
    pl.col('value').mean(),
    pl.len()
])
polars_time = time.time() - start

print(f"Dataset size: {large_n:,} rows")
print(f"Pandas aggregation: {pandas_time*1000:.2f}ms")
print(f"Polars aggregation: {polars_time*1000:.2f}ms")
print(f"Polars speedup: {pandas_time/polars_time:.1f}x")

## Saving Results

Blaze supports COPY TO syntax for exporting results:

In [ ]:
# Example COPY TO queries (for reference)
export_examples = [
    "COPY (SELECT * FROM sales WHERE region = 'North') TO 'north_sales.parquet'",
    "COPY (SELECT * FROM sales) TO 'all_sales.csv' WITH (FORMAT CSV, HEADER true)",
    "COPY (SELECT * FROM sales) TO 'sales.json' WITH (FORMAT JSON)"
]

print("Example COPY TO statements:")
for example in export_examples:
    print(f"  {example}")

# Using PyArrow directly to save
import pyarrow.parquet as pq

# Save as Parquet
# pq.write_table(sales_data, 'sales.parquet')
print("\nTo save with PyArrow:")
print("  pq.write_table(result, 'output.parquet')")

## Summary

This notebook demonstrated:

1. **PyArrow Integration** - Creating and manipulating Arrow tables
2. **Pandas Conversion** - Converting between Arrow and Pandas DataFrames
3. **Polars Conversion** - Using Polars for fast analytics
4. **Visualization** - Creating charts with matplotlib
5. **Arrow IPC** - Working with Arrow's binary format
6. **Window Functions** - Ranking and running totals
7. **Complex Queries** - CTEs, subqueries, and CASE expressions
8. **Performance** - Comparing execution times

When Blaze Python bindings are available, you can use them seamlessly with
the Python data science ecosystem.